数据源是发给富士通的脱敏数据，共计3千余条

应富士通要求，将此批数据，利用当前线上模型的方式，训练模型。获取模型结果之后和富士通的训练结果做对比

按照之前完成的特征工程代码，筛选出有效的字段

In [1]:
import math
import re
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from dateutil.parser import parse

### loan & basic

In [2]:
regex1 = re.compile(r'(^((?!公司).)*银行((?!公司).)*$)|中国邮政储蓄银行股份有限公司')
regex2 = re.compile(r'工商银行|农业银行|中国银行|建设银行|交通银行|招商银行|浦发银行|中信银行|华夏银行|光大银行|民生银行|兴业银行|广发银行|平安银行|浙商银行|渤海银行|恒丰银行')
regex3 = re.compile(r'天津银行|河北银行|张家口银行|承德银行|秦皇岛银行|唐山银行|廊坊银行|保定银行|沧州银行|衡水银行|邢台银行|邯郸银行|'
                    r'晋商银行|大同银行|晋城银行|晋中银行|阳泉市商业银行|长治银行|包商银行|内蒙古银行|鄂尔多斯银行|乌海银行|盛京银行|'
                    r'鞍山银行|抚顺银行|本溪市商业银行|丹东银行|锦州银行|营口银行|营口沿海银行|阜新银行|辽阳银行|铁岭市商业银行|朝阳银行|'
                    r'盘锦市商业银行|葫芦岛银行|吉林银行|龙江银行|上海银行|浙江稠州商业银行|莱商银行|江西银行|徽商银行|杭州银行|江苏银行|'
                    r'南京银行|江苏长江商业银行|苏州银行|温州银行|嘉兴银行|湖州银行|绍兴银行|金华银行|台州商行|浙江泰隆商业银行|宁波银行|'
                    r'福建海峡银行|泉州银行|九江银行|赣州银行|上饶银行|齐鲁银行|齐商银行|烟台银行|潍坊银行|临商银行|威海市商业银行|日照银行|'
                    r'德州银行|东营银行|济宁银行|泰安市商业银行|枣庄银行|中原银行|郑州银行|洛阳银行|平顶山银行|焦作中旅银行|湖北银行|'
                    r'汉口银行|长沙银行|华融湘江银行|广州银行|珠海华润银行|东莞银行|广东华兴银行|广东南粤银行|广西北部湾银行|柳州银行|'
                    r'桂林银行|海南银行|重庆三峡银行|浙江民泰商业银行|哈尔滨银行|贵阳银行|成都银行|泸州市商业银行|攀枝花市商业银行|'
                    r'宜宾市商业银行|乐山市商业银行|南充市商业银行|自贡市商业银行|长城华西银行|遂宁市商业银行|绵阳市商业银行|凉山州商业银行|'
                    r'雅安市商业银行|达州市商业银行|重庆银行|贵州银行|富滇银行|曲靖市商业银行|云南红塔银行|西藏银行|长安银行|西安银行|兰州银行|'
                    r'甘肃银行|青海银行|石嘴山银行|乌鲁木齐银行|昆仑银行|新疆汇和银行|库尔勒银行|哈密市商业银行|大连银行|宁波东海银行|'
                    r'宁波通商银行|厦门银行|厦门国际银行|青岛银行')
regex4 = re.compile(r'村镇银行')
regex5 = re.compile(r'农村信用|农村商业|农村合作')
regex6 = re.compile(r'汇丰中国|东亚银行|渣打银行|荷兰银行|东方汇理银行|中信嘉华银行|外换银行|德意志银行|法国兴业银行|国民银行|韩亚银行|'
                    r'蒙特利尔银行|摩根大通银行|瑞士银行|新韩银行|企业银行|友利银行|花旗中国|瑞穗中国|恒生中国|星展中国|三菱中国|华侨永亨中国|'
                    r'南商中国|大华中国|法巴中国|三井中国|汇理中国|盘谷中国|澳新中国|富邦华|华美中国|正信银行|浦发硅谷银行|星明财务|'
                    r'永丰银行|首都银行|协和银行有限公司|摩根士丹利国际银行|新联商业银行|华商银行|大新银行|中信银行国际|玉山银行')
regex7 = re.compile(r'小额贷款')
regex8 = re.compile(r'消费金融')
regex9 = re.compile(r'信托')
regex10 = re.compile(r'汽车金融')
regex11 = re.compile(r'平安普惠|哈尔滨银行龙青支行')
regex12 = re.compile(r'融资担保')
regex13 = re.compile(r'保险')


def into_time(into_id):
    y = ''
    if re.findall('\d{8}', into_id):
        y = re.findall('\d{8}', into_id)[0]
        try:
            parse(y)
        except ValueError:
            y = '20190601'
    else:
        y = '20190601'
    return parse(y)


def interval_month(start_date, end_date):
    months = 0
    if start_date.day > end_date.day:
        months = (end_date.year - start_date.year)*12 + (end_date.month - start_date.month)
    elif start_date.day <= end_date.day:
        months = (end_date.year - start_date.year)*12 + (end_date.month - start_date.month) + 1
    return months


def pt(data_source, field, fun, name):
    table = pd.pivot_table(data_source, index='loan_id', values=field, aggfunc=fun)
    table.rename(columns={field: name}, inplace=True)
    return table

In [37]:
df_basic = pd.read_csv('./data_4-28/fujitsu_basis5.csv')
df_loan = pd.read_csv('./data_4-28/fujitsu_credit5.csv')
df_label = pd.read_csv('./data_4-28/fujitsu_label5.csv').rename(columns={'CLASS':'LABEL'})
df_basic = pd.merge(df_basic,df_label,on='LOAN_ID',how='inner')
df_basic.columns = df_basic.columns.map(lambda x: x.lower())
df_loan.columns = df_loan.columns.map(lambda x: x.lower())
df_loan = df_loan.astype({'grant_date':'datetime64[ns]', 'expired_date':'datetime64[ns]', 'query_date':'datetime64[ns]'})

In [38]:
df_loan['reporttime'] = df_loan.loan_id.map(into_time)
df_loan['credit_month'] = df_loan.apply(lambda x: interval_month(x.grant_date, x.reporttime) + 1, axis=1)
df_loan['used_month'] = df_loan.apply(lambda x: interval_month(x.grant_date, max(x.expired_date, x.query_date)), axis=1)
df_loan['record_month'] = df_loan.used_month.mask(df_loan.used_month > 60, 60)
df_loan['settle_month'] = df_loan.apply(lambda x: interval_month(x.reporttime, x.expired_date), axis=1)
df_loan['overdue_per'] = (df_loan.years5_overdue / df_loan.record_month).mul(100).round(0)

In [39]:
df_loan.head(2)

,loan_id,institution,grant_date,expired_date,query_date,overdue,years5_overdue,days90_overdue,loan_type,loan_state,account_category_main,reporttime,credit_month,used_month,record_month,settle_month,overdue_per
0,BJYD20180530000502,苏宁消费金融有限公司,2016-11-25,2017-12-01,2018-05-28,0,0,0,其他贷款,已结清,个人消费贷款,2018-05-30,20,19,19,-5,0.0
1,BJYD20180530000502,苏宁消费金融有限公司,2017-12-27,2018-06-15,2018-05-01,0,0,0,其他贷款,正常,个人消费贷款,2018-05-30,7,6,6,1,0.0


In [40]:
# l1贷款总笔数
l1 = pt(df_loan, 'loan_type', 'count', 'loan_total_account')
# l2贷款未结清账户数
l2 = pt(df_loan[df_loan.loan_state != '已结清'], 'loan_type', 'count', 'loan_notsettled_account')
# l3贷款已结清账户数
l3 = pt(df_loan[df_loan.loan_state == '已结清'], 'loan_type', 'count', 'loan_settle_account')
# l4贷款在过去6个月内结清笔数
l4 = pt(df_loan[(df_loan.loan_state == '已结清') & (df_loan.settle_month >= -6)], 'loan_type', 'count', 'loan_settle_past6m')
# l5贷款在过去3个月内结清笔数
l5 = pt(df_loan[(df_loan.loan_state == '已结清') & (df_loan.settle_month >= -3)], 'loan_type', 'count', 'loan_settle_past3m')
# l6贷款在过去1个月内结清笔数
l6 = pt(df_loan[(df_loan.loan_state == '已结清') & (df_loan.settle_month >= -1)], 'loan_type', 'count', 'loan_settle_past1m')
# l7贷款在未来1个月内待结清笔数
l7 = pt(df_loan[(df_loan.loan_state != '已结清') & (df_loan.settle_month < 1)], 'loan_type', 'count', 'loan_settle_next1m')
# l8贷款在未来3个月内待结清笔数
l8 = pt(df_loan[(df_loan.loan_state != '已结清') & (df_loan.settle_month < 3)], 'loan_type', 'count', 'loan_settle_next3m')
# l9贷款在未来6个月内待结清笔数
l9 = pt(df_loan[(df_loan.loan_state != '已结清') & (df_loan.settle_month < 6)], 'loan_type', 'count', 'loan_settle_next6m')
# l10贷款在未来12个月内待结清笔数
l10 = pt(df_loan[(df_loan.loan_state != '已结清') & (df_loan.settle_month < 12)], 'loan_type', 'count', 'loan_settle_next12m')
# l18贷款最早发放日期
l11 = pt(df_loan, 'grant_date', 'min', 'loan_first')
# l19贷款最早发放日期（不含助学贷款）
l12 = pt(df_loan[df_loan.account_category_main != '助学贷款'], 'grant_date', 'min', 'loan_first_notstudent')
# l20最近一笔贷款发放日期
l13 = pt(df_loan, 'grant_date', 'max', 'loan_date_nearest')
# l21最近一笔贷款发放日期（不含助学贷款）
l14 = pt(df_loan[df_loan.loan_type != '个人助学贷款'], 'grant_date', 'max', 'loan_date_nearest_notstudent')
# l22贷款发放机构为银行的笔数(1年内)
l15 = pt(df_loan[(df_loan.institution.str.findall(regex1).str.len() > 0) & (df_loan.credit_month < 12)], 'institution', 'count', 'loan_count_bank')
# l23贷款发放机构为国有大行或股份制银行的笔数(1年内)
l16 = pt(df_loan[(df_loan.institution.str.findall(regex2).str.len() > 0) & (df_loan.credit_month < 12)], 'institution', 'count', 'loan_count_bank_country_stock')
# l24贷款发放机构为城市商业银行的笔数(1年内)
l17 = pt(df_loan[(df_loan.institution.str.findall(regex3).str.len() > 0) & (df_loan.credit_month < 12)], 'institution', 'count', 'loan_count_bank_commerce')
# l25贷款发放机构为村镇银行的笔数(1年内)
l18 = pt(df_loan[(df_loan.institution.str.findall(regex4).str.len() > 0) & (df_loan.credit_month < 12)], 'institution', 'count', 'loan_count_bank_village')
# l26贷款发放机构为农村信用社或商业银行的笔数(1年内)
l19 = pt(df_loan[(df_loan.institution.str.findall(regex5).str.len() > 0) & (df_loan.credit_month < 12)], 'institution', 'count', 'loan_count_bank_rcu_commerce')
# l27贷款发放机构为外资银行的笔数(1年内)
l20 = pt(df_loan[(df_loan.institution.str.findall(regex6).str.len() > 0) & (df_loan.credit_month < 12)], 'institution', 'count', 'loan_count_bank_foreign')
# l28贷款发放机构为小额贷款公司的笔数(1年内)
l21 = pt(df_loan[(df_loan.institution.str.findall(regex7).str.len() > 0) & (df_loan.credit_month < 12)], 'institution', 'count', 'loan_count_mcc')
# l29贷款发放机构为消费金融公司的笔数(1年内)
l22 = pt(df_loan[(df_loan.institution.str.findall(regex8).str.len() > 0) & (df_loan.credit_month < 12)], 'institution', 'count', 'loan_count_cfc')
# l30贷款发放机构为信托公司的笔数(1年内)
l23 = pt(df_loan[(df_loan.institution.str.findall(regex9).str.len() > 0) & (df_loan.credit_month < 12)], 'institution', 'count', 'loan_count_trust')
# l31贷款发放机构为汽车金融公司的笔数(1年内)
l24 = pt(df_loan[(df_loan.institution.str.findall(regex10).str.len() > 0) & (df_loan.credit_month < 12)], 'institution', 'count', 'loan_count_afc')
# l32贷款发放机构为重点监测机构的笔数(1年内)
l25 = pt(df_loan[(df_loan.institution.str.findall(regex11).str.len() > 0) & (df_loan.credit_month < 12)], 'institution', 'count', 'loan_count_check')
# l33贷款为房贷的笔数(1年内)
l26 = pt(df_loan[(df_loan.loan_type.isin(['个人住房贷款', '个人公积金住房贷款', '个人商用房（包括商住两用）贷款'])) & (df_loan.credit_month < 12)], 'loan_type', 'count', 'loan_count_house')
# l36贷款为公积金房贷的笔数(1年内)
l27 = pt(df_loan[(df_loan.loan_type == '个人公积金住房贷款') & (df_loan.credit_month < 12)], 'loan_type', 'count', 'loan_count_afl')
# l42贷款为经营性贷款的笔数(1年内)
l28 = pt(df_loan[(df_loan.account_category_main == '个人消费贷款') & (df_loan.credit_month < 12)], 'loan_type', 'count', 'loan_award_consume_avg')
# l42贷款为经营性贷款的笔数(1年内)
l29 = pt(df_loan[(df_loan.account_category_main == '个人经营性贷款') & (df_loan.credit_month < 12)], 'loan_type', 'count', 'loan_count_operate')
# l45贷款为农户贷款的笔数(1年内)
l30 = pt(df_loan[(df_loan.account_category_main == '农户贷款') & (df_loan.credit_month < 12)], 'loan_type', 'count', 'loan_count_peasant')
# l48贷款为汽车贷款的笔数(1年内)
l31 = pt(df_loan[(df_loan.account_category_main == '个人汽车贷款') & (df_loan.credit_month < 12)], 'loan_type', 'count', 'loan_count_car')
# l51贷款五年内逾期总月数
l32 = pt(df_loan, 'years5_overdue', 'sum', 'loan_overdue_month_5y')
# l52贷款两年内逾期总月数（仅统计两年内发放的贷款）
l33 = pt(df_loan[df_loan.credit_month < 24], 'years5_overdue', 'sum', 'loan_overdue_month_2y')
# l53贷款一年内逾期总月数（仅统计两年内发放的贷款）
l34 = pt(df_loan[df_loan.credit_month < 12], 'years5_overdue', 'sum', 'loan_overdue_month_1y')
# l54贷款五年内90天以上逾期次数
l35 = pt(df_loan, 'days90_overdue', 'sum', 'loan_90overdue_5y')
# l55贷款两年内90天以上逾期次数（仅统计两年内发放的贷款）
l36 = pt(df_loan[df_loan.credit_month < 24], 'days90_overdue', 'sum', 'loan_90overdue_2y')
# l56贷款两年内90天以上逾期次数（仅统计两年内发放的贷款）
l37 = pt(df_loan[df_loan.credit_month < 12], 'days90_overdue', 'sum', 'loan_90overdue_1y')
# l57房贷五年内逾期总月数
l38 = pt(df_loan[df_loan.loan_type.isin(['个人住房贷款', '个人公积金住房贷款', '个人商用房（包括商住两用）贷款'])], 'years5_overdue', 'sum', 'loan_house_overdue_month_5y')
# l58房贷两年内逾期总月数（仅统计两年内发放的贷款）
l39 = pt(df_loan[(df_loan.loan_type.isin(['个人住房贷款', '个人公积金住房贷款', '个人商用房（包括商住两用）贷款'])) & (df_loan.credit_month <= 24)], 'years5_overdue',  'sum', 'loan_house_overdue_month_2y')
# l59房贷两年内逾期总月数（仅统计两年内发放的贷款）
l40 = pt(df_loan[(df_loan.loan_type.isin(['个人住房贷款', '个人公积金住房贷款', '个人商用房（包括商住两用）贷款'])) & (df_loan.credit_month <= 12)], 'years5_overdue',  'sum', 'loan_house_overdue_month_1y')
# l60房贷五年内90天以上逾期次数
l41 = pt(df_loan[df_loan.loan_type.isin(['个人住房贷款', '个人公积金住房贷款', '个人商用房（包括商住两用）贷款'])], 'days90_overdue', 'sum', 'loan_house_90overdue_5y')
# l61房贷两年内90天以上逾期次数（仅统计两年内发放的贷款）
l42 = pt(df_loan[(df_loan.loan_type.isin(['个人住房贷款', '个人公积金住房贷款', '个人商用房（包括商住两用）贷款'])) & (df_loan.credit_month <= 24)], 'days90_overdue', 'sum', 'loan_house_90overdue_2y')
# l62房贷两年内90天以上逾期次数（仅统计两年内发放的贷款）
l43 = pt(df_loan[(df_loan.loan_type.isin(['个人住房贷款', '个人公积金住房贷款', '个人商用房（包括商住两用）贷款'])) & (df_loan.credit_month <= 12)], 'days90_overdue', 'sum', 'loan_house_90overdue_1y')
# l63单张贷款最高逾期比例
l44 = pt(df_loan, 'overdue_per', 'max', 'loan_overdue_highest')
# l64房贷逾期账户数
l45 = pt(df_loan[(df_loan.loan_type.isin(['个人住房贷款', '个人公积金住房贷款', '个人商用房（包括商住两用）贷款'])) & (df_loan.years5_overdue > 0)], 'loan_type', 'count', 'loan_overdue_house')
# l65逾期比例>=10%贷款账户数
l46 = pt(df_loan[df_loan.overdue_per >= 10], 'loan_type', 'count', 'loan_overdue_ratio_10')
# l66逾期比例>=20%贷款账户数
l47 = pt(df_loan[df_loan.overdue_per >= 20], 'loan_type', 'count', 'loan_overdue_ratio_20')
# l67逾期比例>=30%贷款账户数
l48 = pt(df_loan[df_loan.overdue_per >= 30], 'loan_type', 'count', 'loan_overdue_ratio_30')
# l681年内发放的贷款逾期比例>=10%贷款账户数
l49 = pt(df_loan[(df_loan.credit_month < 12) & (df_loan.overdue_per >= 10)], 'loan_type', 'count', 'loan_overdue_10_extend_1y')
# l691年内发放的贷款逾期比例>=20%贷款账户数
l50 = pt(df_loan[(df_loan.credit_month < 12) & (df_loan.overdue_per >= 20)], 'loan_type', 'count', 'loan_overdue_20_extend_1y')
# l701年内发放的贷款逾期比例>=30%贷款账户数
l51 = pt(df_loan[(df_loan.credit_month < 12) & (df_loan.overdue_per >= 30)], 'loan_type', 'count', 'loan_overdue_30_extend_1y')
# l712年内发放的贷款逾期比例>=10%贷款账户数
l52 = pt(df_loan[(df_loan.credit_month < 24) & (df_loan.overdue_per >= 10)], 'loan_type', 'count', 'loan_overdue_10_extend_2y')
# l722年内发放的贷款逾期比例>=20%贷款账户数
l53 = pt(df_loan[(df_loan.credit_month < 24) & (df_loan.overdue_per >= 20)], 'loan_type', 'count', 'loan_overdue_20_extend_2y')
# l732年内发放的贷款逾期比例>=30%贷款账户数
l54 = pt(df_loan[(df_loan.credit_month < 24) & (df_loan.overdue_per >= 30)], 'loan_type', 'count', 'loan_overdue_30_extend_2y')
# l74贷款正常笔数(2年内)
l55 = pt(df_loan[(df_loan.loan_state == '正常') & (df_loan.credit_month < 24)], 'loan_type', 'count', 'loan_count_normal')
# l75贷款结清笔数(2年内)
l56 = pt(df_loan[(df_loan.loan_state == '已结清') & (df_loan.credit_month < 24)], 'loan_type', 'count', 'loan_count_settle')
# l76贷款逾期笔数(2年内)
l57 = pt(df_loan[(df_loan.loan_state == '逾期') & (df_loan.credit_month < 24)], 'loan_type', 'count', 'loan_count_overdue')
# l78贷款转出笔数(2年内)
l58 = pt(df_loan[(df_loan.loan_state == '已转出') & (df_loan.credit_month < 24)], 'loan_type', 'count', 'loan_count_out')
# l79贷款正常笔数(1年内)
l59 = pt(df_loan[(df_loan.loan_state == '正常') & (df_loan.credit_month < 12)], 'loan_type', 'count', 'loan_count_normal_in_1y')
# l80贷款结清笔数(1年内)
l60 = pt(df_loan[(df_loan.loan_state == '已结清') & (df_loan.credit_month < 12)], 'loan_type', 'count', 'loan_count_settle_in_1y')
# l81贷款逾期笔数(1年内)
l61 = pt(df_loan[(df_loan.loan_state == '逾期') & (df_loan.credit_month < 12)], 'loan_type', 'count', 'loan_count_overdue_in_1y')
# l83贷款转出笔数(1年内)
l62 = pt(df_loan[(df_loan.loan_state == '已转出') & (df_loan.credit_month < 12)], 'loan_type', 'count', 'loan_count_out_in_1y')
# l84房贷已还期数
l63 = pt(df_loan[df_loan.loan_type.isin(['个人住房贷款', '个人公积金住房贷款', '个人商用房（包括商住两用）贷款'])], 'used_month', 'sum', 'loan_house_repaymonth')
# l91贷款当前逾期账户数
l64 = pt(df_loan[df_loan.overdue == 1], 'loan_type', 'count', 'loan_overdue_account')
# l114贷款发放机构为微众银行的笔数
l65 = pt(df_loan[df_loan.institution.str.findall(r'微众银行').str.len() > 0], 'institution', 'count', 'loan_count_weixin')
# l115贷款发放机构为浙江网商银行的笔数
l66 = pt(df_loan[df_loan.institution.str.findall(r'网商银行').str.len() > 0], 'institution', 'count', 'loan_count_mayi')
# l116微粒贷使用时长(按月计)
l67 = pt(df_loan[df_loan.institution.str.findall(r'微众银行').str.len() > 0], 'used_month', 'max', 'loan_history_weixin')
# l118借呗/网商贷使用时长(按月计)
l68 = pt(df_loan[df_loan.institution.str.findall(r'网商银行|阿里巴巴').str.len() > 0], 'used_month', 'max', 'loan_history_mayi')
# l120贷款信用时长（不包括助学贷款）
l69 = pt(df_loan[df_loan.loan_type != '助学贷款'], 'credit_month', 'max', 'loan_credit_history')

In [41]:
loan_res = pd.concat([eval('l'+ str(i)) for i in range(1,70)], sort=False, axis=1).reset_index().fillna(0).rename(columns={'index':'loan_id'})

In [42]:
loan_res.to_csv('./res_4-28/loan_result5.csv', index=False)

In [43]:
df_basic.to_csv('./res_4-28/basic&label5.csv', index=False)

## call log

In [44]:
from datetime import timedelta

In [45]:
def interval_month(start_date, end_date):
    diff = ''
    if (end_date - start_date) <= timedelta(7):
        diff = '周'
    elif (end_date - start_date) <= timedelta(30):
        diff = '月'
    else:
        diff = 'xx'
    return diff


def pt(data_source, field, fun, name):
    table_pt = pd.pivot_table(data_source, index='loan_id', values=field, aggfunc=fun)
    table_pt.rename(columns={field: name}, inplace=True)
    return table_pt

In [83]:
df_call = pd.read_csv('./data_4-28/fujitsu_call_log5.csv')
df_call.columns = df_call.columns.map(lambda x: x.lower())
df_call = df_call.astype({'call_time':'datetime64[ns]', 'found_time':'datetime64[ns]'})

In [84]:
df_call.call_way.value_counts()

被叫    1106877
主叫     979423
呼转       2484
其他       1540
Name: call_way, dtype: int64

In [85]:
df_call['time_diff'] = df_call.apply(lambda x: interval_month(x.call_time, x.found_time), axis=1)

In [86]:
df_call['time_value'] = df_call.call_time.map(lambda x: x.time())

In [87]:
pt1 = pt(df_call[df_call.time_diff == '月'], 'mobile_num', 'count', 'call_1m')
pt2 = pt(df_call[(df_call.time_diff == '月') & (df_call.call_way == '主叫')], 'mobile_num', 'count', 'call_1m_z')
pt3 = pt(df_call[(df_call.time_diff == '月') & (df_call.call_way == '被叫')], 'mobile_num', 'count', 'call_1m_b')
pt4 = pt(df_call[df_call.time_diff == '周'], 'mobile_num', 'count', 'call_1w')
pt5 = pt(df_call[(df_call.time_diff == '周') & (df_call.call_way == '主叫')], 'mobile_num', 'count', 'call_1w_z')
pt6 = pt(df_call[(df_call.time_diff == '周') & (df_call.call_way == '被叫')], 'mobile_num', 'count', 'call_1w_b')
pt7 = pt(df_call[(df_call.time_value <= parse('2016-01-01 06:00:00').time()) & (df_call.time_diff == '周')], 'mobile_num', 'count', '6clock')
pt8 = pt(df_call[(df_call.time_diff == '月') & (df_call.call_way == '主叫')], 'call_address', 'nunique', 'call_1m_city_z')
pt9 = pt(df_call[(df_call.time_diff == '月') & (df_call.call_way == '被叫')], 'call_address', 'nunique', 'call_1m_city_b')

In [88]:
call_res = pd.concat([eval('pt' + str(i)) for i in range(1,10)], axis=1, sort=True).fillna(0)

In [89]:
call_res['call_1w_n'] = call_res['6clock'] / call_res['call_1w']

In [90]:
call_res.fillna(0, inplace=True)

In [91]:
call_res.drop('6clock', axis=1).reset_index().rename(columns={'index':'lona_id'}).to_csv('./res_4-28/call_result5.csv', index=False)